In [7]:
import logging
from glob import glob
from pathlib import Path
import scipy.io as sio

from live2p.workers import RealTimeQueue
from live2p.offline import run_plane_offline_multifolder
from live2p.utils import tic, ptoc

# logging setup
# change for more or less information...
caiman_loglevel = logging.INFO
live2p_loglevel = logging.DEBUG

# changes how logs/updates are printed
logformat = '{relativeCreated:08.0f} - {levelname:8} - [{module}:{funcName}:{lineno}] - {message}'
logging.basicConfig(level=caiman_loglevel, format=logformat, style='{') #sets caiman loglevel
logger = logging.getLogger('live2p')
logger.setLevel(live2p_loglevel) # sets live2p debug level

%config InlineBackend.figure_format = 'retina'

In [6]:
load_path = 'e:/outfiles'
load_list = [
    '210517_I147_outfile.mat',
    '210518_I147_outfile.mat',
    '220127_HB120_outfile.mat',
    '220131_HB120_outfile.mat',
    '220131_HB113_outfile.mat'
]

pths = [Path(load_path, i) for i in load_list]
[p.exists() for p in pths]

[True, True, True, True, True]

In [9]:
mat = sio.loadmat(pths[0])
mat.keys()

dict_keys(['__header__', '__version__', '__globals__', 'out', '__function_workspace__'])

In [40]:
date = mat['out'][0]['info'][0]['date'][0][0][0]
mouse = mat['out'][0]['info'][0]['mouse'][0][0][0]
fpath = Path(mat['out'][0]['info'][0]['path'][0][0][0])
epochs = fpath.stem.split('_')
epochs

['1', '2', '3', '4', '5', '6', '7', '8', '9']

In [2]:
# experiment info
# put the makeMasks3D image mat file into the FIRST tiff folder

# root folder where all the tiffs for an expt are located
tiff_root = 'k:/test_data/caiman_scratch/test_data'

# list epoch folder names
epoch_list = ['1', '2']

tiff_folders = ['/'.join([tiff_root, e]) for e in epoch_list]

nplanes = 3 # for running multiple planes
fr = 30

# x_start and x_end need to be the same or larger than what is in mm3d
x_start = 110
x_end = 512-110
# we can auto-determine them now...
# but note: if the stim/vis artifact is in the cropped range, there will be problems
# with motion correction and/or F extraction
# mm3d_path = glob(tiff_folder + '/*.mat')[0]
# x_start, x_end = get_true_mm3d_range(mm3d_path)
# print(f'makeMasks3D range determine to be: {x_start} to {x_end} (pixels)')

# pre-allocated frame buffer, per plane
# make sure this is > than the number of frames in your experiment
max_frames = 30000

n_init = 500

In [3]:
params = {
    'fr': fr,
    'p': 1,  # deconv 0 is off, 1 is slow, 2 is fast
    'nb': 3,  # background compenents -> nb: 3 for complex
    'decay_time': 0.1,  # sensor tau
    'gSig': (5, 5),  # expected half size of neurons in pixels, very important for proper component detection
    'init_method': 'seeded',
    'motion_correct': True,
    'expected_comps': 500,
    'update_num_comps': False,
    'update_freq': 100,
    'niter_rig': 2,
    'pw_rigid': False,
    'dist_shape_update': False,
    'normalize': True,
    'sniper_mode': False,
    'test_both': False,
    'ring_CNN': False,
    'simultaneously': True,
    'use_cuda': False,
}

add_cells = {
    'only_init':True,
    'rf':None,
    'update_num_comps':True
}

# uncomment and use if you want to add cells during the experiment
# note: this may require re-running the CNMF fit to get trace data for detected cells before they were detected
# params = {**params, **add_cells}

In [9]:
# run all planes offline
# as run_plane_offline is already multi-process, so the simplest way to do this is to run planes sequentially

t = tic()
data = []
for p in range(nplanes):
    print(f'***** Starting Plane {p} *****')
    result = run_plane_offline_multifolder(p, tiff_folders, params, x_start, x_end, n_init, max_frames)
    data.append(result)
print('All done!')
ptoc(t, 'Whole thing took')

***** Starting Plane 0 *****
getting files for initialization....


00122539 - DEBUG    - [workers:__init__:44] - Setting up params...
00122541 - INFO     - [params:set:972] - Changing key decay_time in group data from 0.4 to 0.1
00122542 - INFO     - [params:set:972] - Changing key p in group preprocess from 2 to 1
00122542 - INFO     - [params:set:972] - Changing key nb in group init from 1 to 3
00122543 - INFO     - [params:set:972] - Changing key gSig in group init from [5, 5] to (5, 5)
00122544 - INFO     - [params:set:972] - Changing key nb in group spatial from 1 to 3
00122544 - INFO     - [params:set:972] - Changing key p in group temporal from 2 to 1
00122545 - INFO     - [params:set:972] - Changing key nb in group temporal from 1 to 3
00122545 - INFO     - [params:set:972] - Changing key init_method in group online from bare to seeded
00122546 - INFO     - [params:set:972] - Changing key update_num_comps in group online from True to False
00122546 - INFO     - [params:set:972] - Changing key update_freq in group online from 200 to 100
0012254

starting initialization...


00122928 - INFO     - [workers:__init__:214] - Starting live2p worker.
00122930 - INFO     - [workers:initialize_onacid:232] - Starting new OnACID initialization for live2p.
00122931 - DEBUG    - [workers:make_init_mmap:237] - Making init memmap...


Plane 0: Found 99 sources from MM3D...


...

00735730 - INFO     - [online_cnmf:fit_next:769] - Updating Shapes
00740027 - INFO     - [online_cnmf:fit_next:769] - Updating Shapes
00744333 - INFO     - [online_cnmf:fit_next:769] - Updating Shapes
00748633 - INFO     - [online_cnmf:fit_next:769] - Updating Shapes
00752975 - INFO     - [online_cnmf:fit_next:769] - Updating Shapes
00753058 - INFO     - [workers:process_frame_from_queue:358] - Total of 1000 frames processed. (Queue 2)
00753059 - INFO     - [workers:process_frame_from_queue:362] - Average processing time: 16 ms. (59.18 Hz) (Queue 2)
00757276 - INFO     - [online_cnmf:fit_next:769] - Updating Shapes
00761626 - INFO     - [online_cnmf:fit_next:769] - Updating Shapes
00765935 - INFO     - [online_cnmf:fit_next:769] - Updating Shapes
00770233 - INFO     - [online_cnmf:fit_next:769] - Updating Shapes
00774508 - INFO     - [online_cnmf:fit_next:769] - Updating Shapes
00774591 - INFO     - [workers:process_frame_from_queue:358] - Total of 1500 frames processed. (Queue 2)
0077

done!


00966009 - INFO     - [cluster:stop_server:344] - No cluster to stop...
00966011 - INFO     - [cluster:stop_server:358] - stop_cluster(): done
00966012 - DEBUG    - [workers:_stop_cluster:96] - Cluster stopped.
00966013 - DEBUG    - [workers:__del__:74] - Worker object destroyed on delete.


All done!
Whole thing took 845.5544 s


845.5544021999999